# Python Decorators

- Python's answer to C macros, Java annotations, C# attributes, and JavaScript decorators
- Most similar to JavaScript decorators (interpreted runtime behavior)
- Wraps one piece of code with another
- Supports a terse @ syntax for composing higher-order functions
- Defines function that takes another function and returns a new function
- Used to extend behavior of a function without explicitly modifying it
- Used for cross-cutting concerns
- Aspect-Oriented Programming 
- Decorators vs. the Decorator Pattern

## Topics

- Functional Programming
- Simple Decorator Function
- Decorator Syntactic Sugar
- Invocation Counter
- Parameterized Decorator Function
- Function Call Rate Limiting
- Authorization and Access Control
- Logging and Instrumentation
- Stack Trace and Debugging
- The @timing Decorator
- Memoization and Performance Enhancement
- The @functools.wraps Decorator
- Decorator Classes
- The @staticmethod Decorator
- The @staticclassmethod Decorator

## Functional Programming
- Functions
- Function Objects
- Lambdas
- Higher Order Functions

In [1]:
# define function
def my_func_1(x):
    print("my_func_1 called")
    return x**2

# directly call function by name
result = my_func_1 (4)
print(result) # output: 16

# indirectly call function via function object
f = my_func_1
result = f(5)
print(result) # output: 25

# pass function object as a function argument
def call_function(f, x):
    return f(x)
result = call_function(f, 6)
print(result) # output: 36

# return function object as a function result
def return_function(x):
    return lambda : x * x    # lambda expression but could have used a named function
print('calling returned lambda function')
result = return_function(7)()
print(result) # output: 49

my_func_1 called
16
my_func_1 called
25
my_func_1 called
36
calling returned lambda function
49


## Simple Decorator Function

- Decorator function
- Nested return function
- Decorated target function
- Extends functionality via code injection

In [45]:
# simple decorator function
def my_decorator_2(old_func):
    print("my_decorator_2 called")    
    def new_func(n):                     # nested function
        print("new_func called")
        return old_func(n)               # invoke original target function
    return new_func                      # return nested function

# target function to be decoratorated
def my_func_2(x):
    print("my_func_2 called")
    return x**2

result = my_func_2(4)                    # call function before decoratoration
print(result)

print('---')

my_func_2 = my_decorator_2(my_func_2)    # programatically decorate function
result = my_func_2(4)                    # call function after decoratoration
print(result)

my_func_2 called
16
---
my_decorator_2 called
new_func called
my_func_2 called
16


## Decorator Syntactic Sugar

- Uses the declarative @ decorator syntax
- Automatically sets up higher order function logic as shown in previous example
- Extends functionality via code injection

In [3]:
# decorator function
def my_decorator_3(old_func):
    print("my_decorator_3 called")
    def new_func(n):
        print("new_func called")
        return old_func(n)
    return new_func

# decorator syntactic sugar
@my_decorator_3                     # declaratively decorate function
def my_func_3(x):
    print("my_func_3 called")
    return x**2
    
result = my_func_3(4)
print(result)

my_decorator_3 called
new_func called
my_func_3 called
16


# Invocation Counter

- Uses a closure to store persistent fucntion call 
- Fucntion call counter is initialized to zero
- Fucntion call counter is incremented on every call to the function

In [4]:
# invocation counter decorator
def invocation_counter(old_func):
    counter = 0                     # closure
    def new_func(n):
        nonlocal counter
        counter += 1
        print("counter: ", counter)
        return old_func(n)
    return new_func

# decorator syntactic sugar
@invocation_counter
def square(x):
    return x**2
    
result = print(square(4))
result = print(square(5))
result = print(square(6))

counter:  1
16
counter:  2
25
counter:  3
36


## Parameterized Decorator Function

- Parameter is used to provide decorator with initial value
- Any number of parameters of any type could be provided
- The parameter value is stored in a closure for future use
- The parameter value is displayed on subsequent invokations for demonstration purposes
- The decorator is applied to a function, with an argument provided

In [5]:
# parameterized decorator function
def my_decorator_4(text):
    print("my_decorator_4 called with parameter:", text)
    def wrap_old_func(old_func):
        print("wrap_old_func called")
        def new_func(n):
            print("new_func called")
            print(text)
            return old_func(n)
        return new_func
    return wrap_old_func

# decorator syntactic sugar
@my_decorator_4("Hello World!")    # decoratror with parameter
def my_func_4(x):
    print("my_func_4 called")
    return x**2
    
result = my_func_4(4)
print(result)

my_decorator_4 called with parameter: Hello World!
wrap_old_func called
new_func called
Hello World!
my_func_4 called
16


## Function Call Rate Limiting

- A decorator named limit_rate is defined with a parameter named min_time
- A closure is created containing a variable named prev_time initialized as time.clock() - min_time
- Every time the decorated function is called, curr_time is established
- If time transpired is less than min_time then prev_time is updated and None is returned
- Otherwise prev_time is updated and the result of the target function is returned
- If the decorated function is called repeatedly in rapid succession it ignores the call
- If the decorated function is  called repeatedly with a sufficient delay it makes the call

In [47]:
import time

# limit rate decorator
def limit_rate(min_time):
    def wrap_old_func(old_func):
        prev_time = time.clock() - min_time  # closure
        def new_func(n):
            nonlocal prev_time
            curr_time = time.clock()
            print(curr_time - prev_time)
            if curr_time - prev_time < min_time:
                prev_time = curr_time
                return None
            else:
                prev_time = curr_time
                return old_func(n)
        return new_func
    return wrap_old_func

# decorator syntactic sugar
@limit_rate(1.0)                             # consecutive calls must be at least 1 second apart
def square(x):
    return x**2
    
result = print(square(4))    # output: 16
result = print(square(5))    # output: None
result = print(square(6))    # output: None
time.sleep(2.0)
result = print(square(7))    # output: 49
result = print(square(8))    # output: None
result = print(square(9))    # output: None

1.000076304846516
16
0.0009816997771849856
None
0.00047612439084332436
None
2.003317252794659
49
0.00039267991087399423
None
0.00016287291691696737
None


# Authorization and Access Control

- Initially no user is logged in and this authorized is False
- The login function is hard coded for demonstration purposes
- The logout function is hard coded for demonstration purposes
- The require_auth decorator forces its target function to require authentication
- If authorized is True then the DoSomethingDangerous executes its code normally
- If authorized is False then the DoSomethingDangerous does not execute its code

In [48]:
authorized = False

def login(username, password):    
    global authorized
    if username == 'Carole' and password == 'Pa55w.rd':
        authorized = True
        print('login succeeded')
    else:
        print('login failed')

def logout():
    global authorized
    print('logout called')
    authorized = False
    
def require_auth(func):
    def decorated_func(*args, **kwargs):
        if not authorized:
            print('ERROR: Not authorized')
            return None
        return func(*args, **kwargs)
    return decorated_func

@require_auth
def DoSomethingDangerous():
    print('DoSomethingDangerous called with authorized:', authorized)

DoSomethingDangerous()          # output: ERROR: Not authorized
print('---')
login('Carole', 'Pa55w.rd')
DoSomethingDangerous()          #  output: DoSomethingDangerous called with authorized: True
print('---')
logout()
DoSomethingDangerous()          #  output: ERROR: Not authorized
print('---')
login('David', 'wrong-guess')
DoSomethingDangerous()          #  output: ERROR: Not authorized
print('---')
login('Carole', 'Pa55w.rd')
DoSomethingDangerous()          # output: DoSomethingDangerous called with authorized: True

ERROR: Not authorized
---
login succeeded
DoSomethingDangerous called with authorized: True
---
logout called
ERROR: Not authorized
---
login failed
ERROR: Not authorized
---
login succeeded
DoSomethingDangerous called with authorized: True


# Logging and Instrumentation

In [44]:
import datetime
import time

def logging(func):
    def func_with_logging(*args, **kwargs):
        print(func.__name__ + " args: " + str(args) + " kwargs" + str(kwargs) + " called at: " + str(datetime.datetime.now()))
        return func(*args, **kwargs)
    return func_with_logging

@logging
def addition_func(x):
   """Do some math."""
   return x + x

result = addition_func(4)
time.sleep(2)
result = addition_func(5)
time.sleep(5)
result = addition_func(6, 33)
time.sleep(1)
result = addition_func(6, x = 42)

addition_func args: (4,) kwargs{} called at: 2018-04-24 16:35:17.948786
addition_func args: (5,) kwargs{} called at: 2018-04-24 16:35:19.950062
addition_func args: (6, 33) kwargs{} called at: 2018-04-24 16:35:24.964340


TypeError: addition_func() takes 1 positional argument but 2 were given

## Stack Trace and Debugging

In [ ]:
# ???

# The @timing Decorator

In [ ]:
# ???

## Memoization and Performance Enhancement

- Optimization technique that leverages cached results
- Can speed up some algorithms by avoiding costly redundant calculations

In [12]:
def memoize(func):
    cache = func.cache = {}
    def memoized_func(*args, **kwargs):
        key = str(args) + str(kwargs)
        if key not in cache:
            cache[key] = func(*args, **kwargs)
        return cache[key]
    return memoized_func

import time

def timeit(method):
    def timed(*args, **kw):
        t_start = time.time()
        result = method(*args, **kw)
        t_end = time.time()
        print (f"{method.__name__} -> {t_end-t_start} seconds")
        return result
    return timed

@memoize
def fibonacci_memoized(n):    # O(n) -> time complexity is linear (each result for 0 to n only once)
    if n == 0: return 0
    if n == 1: return 1
    else: return fibonacci_memoized(n-1) + fibonacci_memoized(n-2)

def fibonacci_plain(n):       # O(2^n) -> time complexity is exponetial
    if n == 0:return 0
    if n == 1:return 1
    else: return fibonacci_plain(n-1) + fibonacci_plain(n-2)

@timeit
def call_fibonacci_plain(n):
    return fibonacci_plain(n)

print("Wait for result...")

result = call_fibonacci_plain(35)
print("    call_fibonacci_plain(35):", result)

# = call_fibonacci_plain(50)  # takes long time then blows the stack
#print("    call_fibonacci_plain(50):", result)

@timeit
def call_fibonacci_memoized(n):
    return fibonacci_memoized(n)

result = call_fibonacci_memoized(35)
print("    call_fibonacci_memoized(35):", result)

result = call_fibonacci_memoized(35)
print("    call_fibonacci_memoized(35):", result)

result = call_fibonacci_memoized(50)
print("    call_fibonacci_memoized(50):", result)

result = call_fibonacci_memoized(100)
print("    call_fibonacci_memoized(100):", result)

Wait for result...
call_fibonacci_plain -> 11.335016250610352 seconds
    call_fibonacci_plain(35): 9227465
call_fibonacci_memoized -> 0.0 seconds
    call_fibonacci_memoized(35): 9227465
call_fibonacci_memoized -> 0.0 seconds
    call_fibonacci_memoized(35): 9227465
call_fibonacci_memoized -> 0.0 seconds
    call_fibonacci_memoized(50): 12586269025
call_fibonacci_memoized -> 0.0 seconds
    call_fibonacci_memoized(100): 354224848179261915075


## Memoization and Performance Enhancement

In [ ]:
# ???

## The @functools.wraps Decorator

## Decorator Classes

In [ ]:
class email_logit(logit):
    '''
    A logit implementation for sending emails to admins
    when the function is called.
    '''
    def __init__(self, email='admin@myproject.com', *args, **kwargs):
        self.email = email
        super(email_logit, self).__init__(*args, **kwargs)

    def notify(self):
        # Send an email to self.email
        # Will not be implemented here
        pass

In [7]:
__report_indent = [0]

def report(fn):
    """Decorator to print information about a function call for debugging."""
    def wrap(*params,**kwargs):
        call = wrap.callcount = wrap.callcount + 1
        indent = ' ' * __report_indent[0]
        params = ', '.join(
            [a.__repr__() for a in params] + ["{a} = {repr(b)}" for a,b in kwargs.items()]
        )
        fc = f"{fn.__name__}({params})"
        print(f"{indent}{fc} called [#{call}]")
        __report_indent[0] += 1
        ret = fn(*params,**kwargs)
        __report_indent[0] -= 1
        print(f"{indent}{fc} returned {repr(ret)} [#{call}]")

        return ret
    wrap.callcount = 0
    return wrap

#### @report
def mean(*params):
    return sum(params)/len(params)

result = mean(10, 20, 40)
print(result)

@report
def fibonacci(n):
    if n in [0,1]:
        return n
    else:
        return fibonacci(n - 1) + fibonacci(n - 2)

result = fibonacci(4)
print(result)

mean(10, 20, 40) called [#1]


TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [23]:
# Callable Class Decorator
class PrintEnterExit(object):
    def __init__(self, f):
        self.f = f

    def __call__(self):
        print("Entering:", self.f.__name__)
        self.f()
        print("Exited:", self.f.__name__)

@PrintEnterExit
def func1():
    print("In: func1()")

@PrintEnterExit
def func2():
    print("In: func2()")

func1()
func2()

Entering: func1
In: func1()
Exited: func1
Entering: func2
In: func2()
Exited: func2


## The @staticmethod Decorator

In [ ]:
# ???

## The @staticclassmethod Decorator

In [ ]:
# ???